# Advanced Tables for JWST


# Table design goals and requirements

- Mutable container of heterogeneous tabular data
- Relatively lightweight yet powerful enough for most needs
- Responsive to astronomy community needs
  - For JWST community — if something is missing, broken, needs improvement then ASK!
- Deep integration with Astropy (I/O, units, quantity)
- Persistent metadata (column units, table header keywords, formatting)
- Support missing data

# Why doesn't Astropy use Pandas DataFrame?

- Mutable container of heterogeneous tabular data **(only scalar data)**
- **Relatively lightweight** yet powerful enough for most needs
- Responsive to **astronomy community** needs
- **Deep integration with Astropy (I/O, units, quantity)**
- **Persistent metadata (column units, table header keywords, formatting)**
- Support missing data: **Pandas will cast ``int`` types to ``float64`` to use ``NaN``**
  - Large ``int64`` values lose precision
  - Short int (e.g., ``uint8``) values take 4 times as much memory as ``MaskedColumn``

### Nevertheless...

We recognize Pandas is very fast, powerful, and widely used.

*The Astropy Project recommendation is to use `astropy.Table` where possible. This especially applies to community packages.*


# Example: multiband photometry of a field

### Observations in five bands of a single field with five "galaxy-like" sources

- Assumes basic image reduction and source detection is done
- Could be similar to JWST post-image processing workflow

### Key Table concepts to be covered

- Basic table structure (dict of independent column objects)
- Base column class properties and attributes (flexibility in data elements)
- Table mutability and formatting
- Database operations: join, grouping, binning, stacking, indexing
- Missing data
- Mixin columns (Quantity, Time, Coordinates, QTable vs. Table)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import astropy
from astropy import table
from astropy.table import Column, Table, QTable
from astropy.time import Time
import astropy.units as u
import photutils.datasets
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
astropy.__version__

### Make a table of  five random Gaussians corresponding to fake elliptical galaxy-like sources

Use a [photutils utility function](https://photutils.readthedocs.io/en/stable/api/photutils.datasets.make_random_gaussians_table.html) `make_random_gaussians_table`.

In [ ]:
n_sources = 5
param_ranges = dict([('flux', [1000, 10000]),
                ('x_mean', [10, 190]),
                ('y_mean', [10, 190]),
                ('x_stddev', [2, 5]), 
                ('y_stddev', [2, 5]),
                ('theta', [0, np.pi])])
sources = photutils.datasets.make_random_gaussians_table(n_sources, param_ranges, seed=1)

In [ ]:
sources

### Digression: learn a little about the Table and Column objects

- `Table` is a **container class** where `Table.columns` is the main table data structure
- `Table.columns` is an OrderedDict of columns (`Column`, `MaskedColumn`, or mixin column)
- `Column` class inherits from `np.ndarray`
- `MaskedColumn` class inherits from `np.ma.MaskedArray`

In [ ]:
type(sources.columns).__mro__

In [ ]:
type(sources.columns['flux']).__mro__

In [ ]:
sources.columns['flux'] is sources['flux']

### Make a synthetic image for cutouts

In [ ]:
img = photutils.datasets.make_gaussian_sources_image(shape=(200, 200), 
                                               source_table=sources)
plt.imshow(img);  # Trick: trailing semicolon to suppress output

### Make postage-stamp cutouts for each source

In [ ]:
# Make integer columns with rounded representation of source mean position 

sources['x0'] = np.round(sources['x_mean']).astype(int)
sources['y0'] = np.round(sources['y_mean']).astype(int)

In [ ]:
# Generate list of cutout images around each source

npix = 10
cutouts = [img[y0-npix:y0+npix, x0-npix:x0+npix] for x0, y0 in sources['x0', 'y0']]

# `cutouts` is a list of 2-d ndarrays

In [ ]:
# Add the cutouts into table: each element is a 2-d image
# This shows:
#  - Table mutability and independent columns
#  - Storage of ndarray in each table cell

sources['cutout'] = cutouts

In [ ]:
plt.imshow(sources['cutout'][3], interpolation='nearest');

### Add a source identifier to the `Table`

In [ ]:
sources['id'] = ['jwst-{}-{}'.format(x0, y0) for x0, y0 in sources['x0', 'y0']]
sources

### Formatting: let's be a little fussy about the `Table`
- Put the 'id' column first
- Make the precision of table outputs more reasonable (and beautiful!)
- Add units to `flux` and `theta` columns



In [ ]:
# Move the `id` column to be the first column 
# (Should Table get a method `move_column` to make this easier?)
sources_id = sources['id']
del sources['id']
sources.add_column(sources_id, index=0)

# Set the output formatting for particular columns
for name in ('flux', 'x_mean', 'y_mean', 'x_stddev', 'y_stddev', 'theta'):
    sources[name].format = '.3f'  # Could also use '%.3f' or '{:.3f}'
sources['cutout'].format = '.3g'

sources

### Digression: table and column summary information

In [ ]:
sources.info

In [ ]:
sources.info('stats')
# You can write your own info specifications!

In [ ]:
# You can roll your own custom info!

from astropy.utils.data_info import data_info_factory
mystats = data_info_factory(names=['my_min', 'my_median', 'my_max'],
                            funcs=[np.min, np.median, np.max])
sources.info(mystats)

#### Column info: name, dtype, unit, format, description

In [ ]:
sources['theta'].info.description = 'Elliptical gaussian rotation angle'
sources['theta'].info

### Make fake observations of these sources in five bands 'u', 'b', 'v', 'r', 'k'

In [ ]:
def make_observation(sources, band, flux_mult):
    """
    Make fake observation of ``sources`` in a field in ``band``.  Apply
    ``flux_mult`` flux multiplier and some gaussian noise on parameters.
    """
    n = len(sources)
    out = sources.copy()

    # Multiply flux by randomized version of flux_mult
    out['flux'] *= flux_mult * np.random.normal(loc=1, scale=0.1, size=n)

    # Add 1.0 pixel of position and stddev noise
    for name in ('x_mean', 'y_mean'):
        out[name] += np.random.normal(loc=0, scale=1.0, size=n)
  
    # Add 0.1 pixel noise to stddev
    for name in ('x_stddev', 'y_stddev'):
        out[name] += np.random.normal(loc=0, scale=0.1, size=n)
        
    # Add a list that repeats the ``band`` as the second column
    out.add_column(Column([band] * n, name='band'), index=1)
    
    # Make integer columns with rounded representation of source mean position  
    sources['x0'] = np.round(sources['x_mean']).astype(int)
    sources['y0'] = np.round(sources['y_mean']).astype(int)
    
    # Generate list of cutout images around each source
    img = photutils.datasets.make_gaussian_sources_image(shape=(200, 200), source_table=out)
    npix = 10
    cutouts = [img[y0-npix:y0+npix, x0-npix:x0+npix] for x0, y0 in sources['x0', 'y0']]
    sources['cutout'] = cutouts

    return out

In [ ]:
sources_list = []
for band, flux_mult in [('u', 0.1), 
                        ('b', 0.2), 
                        ('v', 0.5), 
                        ('r', 1.0), 
                        ('k', 1.5)]:
    sources_list.append(make_observation(sources, band, flux_mult))
    
sources_list[0]

# Notice that the formatting and units got inherited into our new tables

In [ ]:
# Pretend that there are non-detections in some bands

sources_list[0].remove_rows([1,2,3,4])
sources_list[1].remove_row(1)
sources_list[3].remove_row(0)

## Database-like features for more power: vstack, indexing, group, and join

Our list of source tables **`sources_list`** has the raw data we need for analysis but is inconvenient.  Things we'd like to do:

- Find all observations of a particular source
- List all sources in a particular band
- Compute statistics for a particular source (mean centroid, mean image cutout)
- Make a single wide table organized by source

See [Table high-level operations](http://docs.astropy.org/en/stable/table/operations.html) for all of the details.

### Stacking

In [ ]:
# Stack the list of tables to create a single table (database) of every source observation.

srcs = table.vstack(sources_list)

srcs

### Indexing

- Indexing means that supplemental information (an index) is added to the table that allows access to particular elements in `time << O(N)`.  
- In the case of astropy `Table`, it uses a binary search of an ordered index table `O(log(N))`.

In [ ]:
# Now add a database index on the `id` column.  This becomes the 'primary key'.
# In this case it does not need to be unique, though one can declare that an
# index must be unique.

srcs.add_index('id')

In [ ]:
# Now access elements with id == 'jwst-84-69'.  This should be familiar to Pandas users.
# This returns another Table.

srcs.loc['jwst-84-69']

In [ ]:
# Let's make a secondary index to allow slicing the table by band
srcs.add_index('band')

In [ ]:
# Get a table of all 'b' band source detections

srcs.loc['band', 'b']

In [ ]:
# A special case is if only one table row is selected, in which case
# a Row object is returned.  This is convenient for the common use case
# of a table with unique keys.

srcs.loc['band', 'u']

### Digression: difference between `Row` and length=1 `Table`

- Indexing a single element of a table returns a `Row` object which can be used to set or access a column value.  This always returns a scalar value.
- Indexing a single row slice of a table returns a `Table`, so accessing a column returns a `Column` object (an array) with a length of 1.
- This is consistent with NumPy structured arrays and Pandas (`df.ix[0]` vs. `df[0:1]`).

In [ ]:
srcs[0]['band']

In [ ]:
srcs[0:1]['band']

### Digression: table access performance

**``srcs[0:1]['band']``** and **``srcs['band'][0:1]``** give the same output, but performance is very different!

In [ ]:
# This creates an entire new Table object (slow) and then selects one column (fast)
%timeit srcs[0:1]['band']

In [ ]:
# This selects a column (fast) and then slices it (fast-ish, creates new Column)
%timeit srcs['band'][0:1]

In [ ]:
# For the most performance, drop the `Column` machinery (no metadata) and use a straight numpy array

%timeit np.array(srcs['band'], copy=False)[0:1]

### Grouping

Astropy `Table` supports the powerful concept of grouping which lets you group the rows into sub-tables which you can then:

- [Examine](http://docs.astropy.org/en/stable/table/operations.html#manipulating-groups): select and loop over groups
- [Aggregate](http://docs.astropy.org/en/stable/table/operations.html#aggregation): apply a reduction function like `np.mean` to each group
- [Filter](http://docs.astropy.org/en/stable/table/operations.html#filtering): select groups by means of a selection function

This is a close cousin to indexing, and if a table is already indexed then creating the grouped version is faster.

In [ ]:
srcs_grouped = srcs.group_by('id')

# srcs_grouped has all the same rows but now ordered by ``id``
srcs_grouped

In [ ]:
for src in srcs_grouped.groups:
    print(src)

In [ ]:
# Now let's make a new table where each row is the mean of all rows in the group
mean_srcs = srcs_grouped.groups.aggregate(np.mean)
mean_srcs

In [ ]:
# We can define custom behavior depending on column type or even name

def sources_mean(arr):
    if arr.dtype.kind in ('S', 'U'):
        out= ', '.join(arr)
    elif arr.info.name == 'flux':
        # Take the log mean
        out = np.exp(np.mean(np.log(arr)))
    else:
        out = np.mean(arr, axis=0)

    return out

In [ ]:
mean_srcs = srcs_grouped.groups.aggregate(sources_mean)
mean_srcs

In [ ]:
plt.imshow(mean_srcs['cutout'][3], interpolation='nearest')

### Digression: binning

A common tool in analysis is to **bin** a table based on some reference value. Examples:

- Photometry of a binary star in several bands taken over a span of time which should be binned by orbital phase.
- Reducing the sampling density for a table by combining 100 rows at a time.
- Unevenly sampled historical data which should binned to four points per year.

The common theme in all of these cases is to convert the key value array into a 
new `float`- or `int`-valued array whose values are identical for rows in the same output bin. As an example, generate a fake light curve:


In [ ]:
year = np.linspace(2000.0, 2010.0, 200)  # 200 observations over 10 years
period = 1.811
y0 = 2005.2
mag = 14.0 + 1.2 * np.sin(2 * np.pi * (year - y0) / period) + np.random.normal(scale=0.1, size=200)
phase = ((year - y0) / period) % 1.0
dat = Table([year, phase, mag], names=['year', 'phase', 'mag'])

In [ ]:
plt.figure(figsize=(8, 2))
plt.subplot(1, 2, 1)
plt.plot(dat['year'], dat['mag'], '.')
plt.xlabel('year')
plt.subplot(1, 2, 2)
plt.xlabel('phase')
plt.plot(dat['phase'], dat['mag'], '.');

In [ ]:
phase_bin = np.trunc(phase / 0.1)
phase_bin[:50]

In [ ]:
dat_grouped = dat.group_by(phase_bin)
dat_mean = dat_grouped.groups.aggregate(np.mean)
dat_std = dat_grouped.groups.aggregate(np.std)

In [ ]:
plt.figure(figsize=(4, 2))
plt.xlabel('phase')
plt.errorbar(x=dat_mean['phase'], xerr=0.05, y=dat_mean['mag'], yerr=dat_std['mag'], fmt='.');

### Join tables to make a single wide table by source `id`

- Have one row corresponding to each of the five sources
- Each row has columns with the five band u, b, v, r, k properties
- De-duplicate column names by labeling as `{colname}_{band}`

Because there are non-detections for some bands/sources, the result is a **Masked Table**.

In [ ]:
sources_id = None
for left, right in zip(sources_list[:-1], sources_list[1:]):
    sources_id = table.join(left=sources_id or left,
                            right=right,
                            keys='id',
                            join_type='outer',
                            table_names=[left['band'][0], right['band'][0]])
sources_id

In [ ]:
# Inspect a masked element

sources_id.add_index('id')
sources_id.loc['jwst-84-69']['flux_u'] is np.ma.masked

## Astropy integration: quantities, units, mixin columns, `QTable` and all that

A major feature of astropy `Table` is integrated support for:

- ``Quantity`` columns that have meaningful units
- ``Time`` and ``Coordinate`` columns
- Other mixin columns

Mixin columns are object types that adhere to the mixin protocol and are
stored and manipulated **natively** in the table.

### Example: store a `Time` object in a table

In [ ]:
t = Table()
t['index'] = [1, 2]
t['time'] = Time(['2001-01-02T12:34:56', '2001-02-03T00:01:02'])
t

In [ ]:
# The time column is a bona-fide Time object
t['time']

In [ ]:
# In case you don't believe me
t['time'].mjd

### Quantity: doesn't `Table` already support units?

We saw in the `sources` table that we can define units.  Aren't we good to go?  **Not quite!**

In [ ]:
type(sources['theta'])

In [ ]:
sources['theta'].unit

**The normal table `Column` class is carrying `unit` as an attribute.**

It is no more special than `description` or `format`:

In [ ]:
t2 = sources['theta'] ** 2
t2.unit

### `QTable` to the rescue

Astropy has a `QTable` class for tables that use `Quantity` objects for columns with units.

See the [Quantity and QTable](http://docs.astropy.org/en/stable/table/mixin_columns.html#quantity-and-qtable) section for more details.

In [ ]:
# Let's make `flux` and `theta` be real Quantity objects!
qsources = QTable(sources)
qsources

*The repeated presence of `electron` and `rad` in each Quantity value is a problem that is fixed in 1.3-dev*.

In [ ]:
type(qsources['theta'])

In [ ]:
qt2 = qsources['theta'] ** 2
qt2.unit

### Summary of `Table` and `QTable`

In short, `Table` and `QTable` are **identical in every way except for handling columns with units**:
- `Table` uses `Column` for any columns with units (with informational-only unit attribute)
- `QTable` uses `Quantity` for any columns with units (with meaningful unit attribute)

Use `QTable` in general if you are fully on-board with using `Quantity` and do not deal with much legacy code.

Use `Table` if you are using code that is not `Quantity`-aware, OR if you need full missing data support.  `Quantity` does not support missing (masked) data.

### Digression: storing a Pandas Series within Astropy `Table`

In [ ]:
from astropy.utils.data_info import ParentDtypeInfo
import pandas as pd

class SeriesMixin(pd.Series):
    info = ParentDtypeInfo()

    def __init__(self, *args, **kwargs):
        name = kwargs.pop('name')
        super().__init__(*args, **kwargs)
        self.info.name = name

In [ ]:
s = SeriesMixin((np.arange(5)-2)**2, name='s')
pt = Table([s])

In [ ]:
pt['s'].info

In [ ]:
isinstance(pt['s'], pd.Series)

In [ ]:
pt['s'].plot();

## But wait, there's more... for another day

- Transform to/from other table-like objects (Pandas DataFrame, NumPy structured array, LSST)
- Subclass for custom applications (documentation example of params)
- Unified interface for I/O (ASCII, FITS, votable)